In [1]:
#Import pandas, matplotlib.pyplot, and seaborn in the correct lines below
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

from library.sb_utils import save_file

In [2]:
# CSV data file is the raw_data directory
whisky_data_init = pd.read_csv('../raw_data/scotch_review.csv')

In [3]:
display(whisky_data_init.shape)

(2247, 7)

In [4]:
#Call the info method on whisky data to see a summary of the data
whisky_data_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247 entries, 0 to 2246
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2247 non-null   int64 
 1   name          2247 non-null   object
 2   category      2247 non-null   object
 3   review.point  2247 non-null   int64 
 4   price         2247 non-null   object
 5   currency      2247 non-null   object
 6   description   2247 non-null   object
dtypes: int64(2), object(5)
memory usage: 123.0+ KB


In [5]:
#Check for Null values
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#whisky_data as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([whisky_data_init.isnull().sum(), 100 * round((whisky_data_init.isnull().mean()),3)], axis=1)
missing.columns=['Count', '%']
missing.sort_values(by='Count', ascending = False)


,Count,%
Unnamed: 0,0,0.0
name,0,0.0
category,0,0.0
review.point,0,0.0
price,0,0.0
currency,0,0.0
description,0,0.0


In [6]:
#Call the head method on whisky data to print the first several rows of the data
whisky_data_init.head()

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


In [7]:
# Check if multiple currencies
whisky_data_init.currency.unique()

array(['$'], dtype=object)

In [8]:
# Check if multiple categories
whisky_data_init.category.unique()

array(['Blended Scotch Whisky', 'Single Malt Scotch',
       'Blended Malt Scotch Whisky', 'Grain Scotch Whisky',
       'Single Grain Whisky'], dtype=object)

In [9]:
# Check and drop redundant columns
for col in whisky_data_init.columns:
    if len(whisky_data_init[col].unique()) < 3:
            display(whisky_data_init[col].unique())
whisky_data_init.drop(columns=['Unnamed: 0', 'currency'], inplace=True)

array(['$'], dtype=object)

In [10]:
# Find and replace comma separators in price column
whisky_data_init['price']=whisky_data_init['price'].str.replace(',','')

In [11]:
# Find all the rows wher the 'price' column contains alphabetical characters
pattern = r'[A-Za-z]'
whisky_data_init[whisky_data_init['price'].str.contains(pattern)]['price']

19      $15000 or $60000/set
95                 60000/set
410                60000/set
576                 44/liter
1000               60000/set
1215               60000/set
Name: price, dtype: object

In [12]:
# Default bottle size is 750ml. Normalize prices to 750ml bottles.
# Only 6 exceptions with inconsistent format. 
# Adjust individually, not the most elegant but in this case the most effective way.

In [13]:
whisky_data_init.iloc[19, 3] = 15000
whisky_data_init.iloc[95, 3] = 15000
whisky_data_init.iloc[410, 3] = 15000
whisky_data_init.iloc[576, 3] = 44*0.75
whisky_data_init.iloc[1000, 3] = 15000
whisky_data_init.iloc[1215, 3] = 15000

In [14]:
# Convert price to float
whisky_data_init['price'] = whisky_data_init['price'].astype('float')

In [15]:
# Rename columns to reflect normalization and simplification  
whisky_data_init.rename(columns={"review.point": "points","price": "price ($/750ml)"},inplace=True)

In [16]:
# Review Data after intial cleaning
whisky_data_init.info()
whisky_data_init.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247 entries, 0 to 2246
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             2247 non-null   object 
 1   category         2247 non-null   object 
 2   points           2247 non-null   int64  
 3   price ($/750ml)  2247 non-null   float64
 4   description      2247 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 87.9+ KB


,name,category,points,price ($/750ml),description
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225.0,"Magnificently powerful and intense. Caramels, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.0,What impresses me most is how this whisky evol...
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.0,There have been some legendary Bowmores from t...
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325.0,With a name inspired by a 1926 Buster Keaton m...
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160.0,"Captivating, enticing, and wonderfully charmin..."


In [17]:
# Check and count duplicate names
whisky_data_init.duplicated('name').sum()

24

In [18]:
# List duplicate names
whisky_data_init[whisky_data_init.duplicated(subset=['name'], keep=False)].sort_values('name')

,name,category,points,price ($/750ml),description
983,"Aberlour 12 year old, 40%",Single Malt Scotch,87,55.0,Matured in a mix of bourbon and sherry casks. ...
1613,"Aberlour 12 year old, 40%",Single Malt Scotch,84,44.0,Somehow this 12 year old—a mix of sherry and A...
180,"Aberlour 18 year old, 43%",Single Malt Scotch,92,125.0,This has a significant proportion of sherry wo...
286,"Aberlour 18 year old, 43%",Single Malt Scotch,91,100.0,There was an Aberlour 18 on the market for a s...
594,"Arran 18 year old, 46%",Single Malt Scotch,89,100.0,Following the 2015 limited edition release of ...
593,"Arran 18 year old, 46%",Single Malt Scotch,89,120.0,This expression is the culmination of Arran’s ...
1442,"Benromach 15 year old, 43%",Single Malt Scotch,85,100.0,"A new member of the core range, this has a cin..."
1009,"Benromach 15 year old, 43%",Single Malt Scotch,87,100.0,The aim with Benromach was always to rediscove...
303,"Bowmore, 18 year old, 43%",Single Malt Scotch,91,85.0,This replaces the 17 year old in the standard ...
352,"Bowmore, 18 year old, 43%",Single Malt Scotch,91,85.0,This replaces the 17 year old in the standard ...


In [19]:
# Calculate mean review value and mean price for duplicates (could be processed differently)
whisky_data_init['rev_points'] = whisky_data_init.groupby('name')['points'].transform('mean')
whisky_data_init['rev_price ($/750ml)'] = whisky_data_init.groupby('name')['price ($/750ml)'].transform('mean')

In [20]:
# Drop the duplicate names (keep the first each time)
whisky_data_init.drop_duplicates(subset=['name'], keep='first', inplace=True)

In [21]:
# Drop the old columns
whisky_data_init.drop(columns=['price ($/750ml)', 'points'], inplace=True)

In [22]:
# Rename affected columns to original labels
whisky_data_init.rename(columns={'rev_price ($/750ml)': 'price ($/750ml)', 'rev_points': 'points'}, inplace=True)

In [23]:
# Verify no duplicates left
whisky_data_init.duplicated('name').sum()

0

In [24]:
# Extract additional parameters (code adopted from Orfanakis Konstantinos)
# Age converted to float
whisky_data_init['age'] = whisky_data_init['name'].str.extract(r'(\d+) year')[0].astype(float)
# Remove ABV from Names
whisky_data_init['name'] = whisky_data_init['name'].str.replace(' ABV ', '')
# Alcohol %
whisky_data_init['alcohol%'] = whisky_data_init['name'].str.extract(r"([\(\,\,\'\"\’\”\$] ? ?\d+(\.\d+)?%)")[0]
whisky_data_init['alcohol%'] = whisky_data_init['alcohol%'].str.replace("[^\d\.]", "").astype(float)
# Cost per review point rounded to 2 digits
whisky_data_init['cost_per_point'] = round(whisky_data_init['price ($/750ml)']/whisky_data_init['points'],2)

In [25]:
# Reveiw data
whisky_data_init.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2223 entries, 0 to 2246
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             2223 non-null   object 
 1   category         2223 non-null   object 
 2   description      2223 non-null   object 
 3   points           2223 non-null   float64
 4   price ($/750ml)  2223 non-null   float64
 5   age              1197 non-null   float64
 6   alcohol%         2206 non-null   float64
 7   cost_per_point   2223 non-null   float64
dtypes: float64(5), object(3)
memory usage: 156.3+ KB


In [26]:
# Reveiw and display data for 10 random whiskies
whisky_data_init.sample(10)

,name,category,description,points,price ($/750ml),age,alcohol%,cost_per_point
1137,"Kilchoman 100% Islay (distilled 2008, bottled ...",Single Malt Scotch,The only Islay distillery not on the coast — a...,87.0,100.0,NaN,50.0,1.15
662,"Glenrothes Vintage Reserve, 40%",Single Malt Scotch,Glenrothes is always this fascinating mix of t...,88.0,57.5,NaN,40.0,0.65
1058,"GlenDronach Sauternes Finish, 46%, 14 year old",Single Malt Scotch,"The rich, sweet Sauternes dessert wine adds it...",87.0,75.0,14.0,46.0,0.86
392,"Royal Salute 21 year old, 40%",Blended Scotch Whisky,"Not exactly a new whisky for the Jubilee, just...",90.0,170.0,21.0,40.0,1.89
1905,Douglas Laing Old Particular (distilled at Inv...,Single Grain Whisky,Sweet top notes of brioche loaf with baking sp...,83.0,110.0,28.0,56.5,1.33
1520,"Littlemill 21 year old Burns Malt, 55.1%",Single Malt Scotch,Online retailer TheWhiskyBarrel.com has releas...,85.0,72.0,21.0,55.1,0.85
166,"Compass Box, The Peat Monster, Reserve Edition...",Blended Malt Scotch Whisky,Compass Box Whiskies celebrates the fifth anni...,92.0,150.0,NaN,48.9,1.63
129,"Lagavulin 1995 Feis Ile 2014 bottling, 54.7%",Single Malt Scotch,"A sherry-cask Lagavulin, this immediately show...",93.0,167.0,NaN,54.7,1.80
668,Murray McDavid,Single Malt Scotch,"Yellow gold color. Fresh, floral aromas (remin...",89.0,150.0,NaN,NaN,1.69
364,Duncan Taylor (distilled at Royal Lochnagar) 2...,Single Malt Scotch,"Soft, soothing, and gentle. Layered fruit (bri...",91.0,200.0,23.0,46.3,2.20


In [27]:
# Save initial data set to final data_frame and verify data is identical to cleaned intial data
whisky_data_final = whisky_data_init.copy()
whisky_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2223 entries, 0 to 2246
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             2223 non-null   object 
 1   category         2223 non-null   object 
 2   description      2223 non-null   object 
 3   points           2223 non-null   float64
 4   price ($/750ml)  2223 non-null   float64
 5   age              1197 non-null   float64
 6   alcohol%         2206 non-null   float64
 7   cost_per_point   2223 non-null   float64
dtypes: float64(5), object(3)
memory usage: 156.3+ KB


In [28]:
# Export csv file of cleaned final data set
whisky_data_final.to_csv('whisky_data_final.csv')